In [10]:
import pandas as pd 
import lancedb
import numpy as np 
import os 
import re 
from sentence_transformers import SentenceTransformer
from lancedb.rerankers import CrossEncoderReranker

In [11]:
data_dir = "../../european-city-data/data-sources/"
wikivoyage_docs_dir = data_dir + "wikivoyage/"
wikivoyage_listings_dir = wikivoyage_docs_dir + "listings/"
database_dir = "../database/"

In [14]:
def embed_query(query):
    model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
    # vector_dimension = model.get_sentence_embedding_dimension()   
    embedding = model.encode(query).tolist()
    return embedding

def get_wv_context(query, limit=10, reranking = 0):
    uri = database_dir + "wikivoyage-docs-db"
    db = lancedb.connect(uri)
    print("Connected to DB.")

    query_embedding = embed_query(query)
    table = db.open_table("wikivoyage_documents")

    if reranking: 
        reranker = CrossEncoderReranker()
        results = table.search(query).rerank(reranker=reranker).limit(limit).to_list()

    else:
        results = table.search(query_embedding).limit(limit).to_list()
        
    print("Found the most relevant documents.")
    city_lists = [f"city: {r['city']}, country: {r['country']}, section: {r['section']}, text: {r['text']}" for r in results]

    # context = [f"city: {r['city']}, country: {r['country']}, name: {r['title']}, description: {r['description']}" for r in results]

    return city_lists

In [15]:
query = "Cities in Europe with a great nightlife and art museums"
# query_embedding = embed_query(query)

context = get_wv_context(query, reranking=1)

Connected to DB.


ImportError: Please install tantivy-py `pip install tantivy` to use the full text search feature.

In [8]:
context

["city: Varna, country: Bulgaria, section: Drink, text: The beach at night tends to have the most night life in the summer.\nThere are also clubs and bars in the city but some of them are closed during summer: e.g. '''[https://m.facebook.com/bar.cubo Cubo]''', Horizont, Planet Club.\n",
 'city: Braga, country: Portugal, section: Drink, text: === Nightlife ===\nNightlife in Braga may be quite monotonous sometimes. Clubs in town are open only on weekends and Wednesdays, since it is "academic night" and most students go on having a drink and end the night in one of the local clubs. There are lots of small bars around the historic city centre. Most of them are well hidden on the tight streets, and you will only find them with the help of locals. It is not advised to walk around the city centre at night by yourself. It may look safe and very crowded during the day, but it\'s quite abandoned and very very dark during the night as most people prefer to go into the bars and coffee shops.\n\n',